<a href="https://colab.research.google.com/github/gab-palmeri/aml-geolocalization/blob/sam/step_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The purpose of this notebook is analyze the performance applying the following transformations to the images:
- random horizontal flip
- isotropic resize

# pip install requirements

Remember to click on "Restart Runtime" before go on

In [ ]:
# CosPlace requirements
!pip3 install "faiss_cpu>=1.7.1"
!pip3 install "numpy>=1.21.2"
!pip3 install "Pillow>=9.0.1"
!pip3 install "scikit_learn>=1.0.2"
!pip3 install "torch>=1.8.2"
!pip3 install "torchvision>=0.9.2"
!pip3 install "tqdm>=4.62.3"
!pip3 install "utm>=0.7.0"
!pip3 install "timm"

import torch
#use GPU if available 
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") #'cpu' # 'cuda' or 'cpu'
print(DEVICE)

# Download Datasets and previous data

Downloading with gdown doesn't work properly.

Prefer always to use drive / mount

In [ ]:
import os
import gdown
from google.colab import drive

def download(id, output=None, quiet=True):
  gdown.download(
    f"https://drive.google.com/uc?export=download&confirm=pbef&id={id}",
    output=output,
    quiet=quiet
  )


use_mount = True
resume_model = False

if use_mount:
  drive.mount('/content/drive')

# TOKYO-XS DATASET
if not os.path.isdir("/content/tokyo_xs"):
  if use_mount:
    !jar xvf "/content/drive/MyDrive/Project 6 - Dataset/tokyo-xs.zip"
  else:
    id = "1fBCnap5BRh36474cVkjvjlC-yUTEb1n3"
    download(id, quiet=False)                           # download from our gdrive
    !jar xvf "/content/tokyo-xs.zip"                    # unzip
    !rm -r "/content/tokyo-xs.zip"                      # remove .zip file

if not os.path.isdir("/content/tokyo_xs"):
  raise FileNotFoundError(f"Can't download tokyo xs")

#TOKYO NIGHT DATASET
if not os.path.isdir("/content/tokyo-night"):
  if use_mount:
    !jar xvf "/content/drive/MyDrive/Project 6 - Dataset/tokyo-night.zip"
  else:
    id = "1tbEQL4XrUPaHyK5_OF16cpvam80PyttR"
    download(id, quiet=False)                           # download from our gdrive
    !jar xvf "/content/tokyo-night.zip"                    # unzip
    !rm -r "/content/tokyo-night.zip"

if not os.path.isdir("/content/tokyo-night"):
  raise FileNotFoundError(f"Can't download tokyo night")

# SAN FRANCISCO - XS DATASET
if not os.path.isdir("/content/small"):
  if use_mount:
    !jar xvf "/content/drive/MyDrive/Project 6 - Dataset/sf-xs.zip"
  else:
    id = "1brIxBJmOgvuzFbI57f5LxnMxjccUu993"
    download(id, quiet=False)                           # download
    !jar xvf "/content/sf-xs.zip"                       # unzip
    !rm -r "/content/sf-xs.zip"                         # remove .zip file

if not os.path.isdir("/content/small"):
  raise FileNotFoundError(f"Can't download sfxs")

# resumed model
if resume_model and not os.path.isfile("/content/saved_models/arcface_model.pth"):
  if use_mount:
    !jar xvf "/content/drive/MyDrive/Project 6 - Dataset/saved_models.zip"

if resume_model and not os.path.isfile("/content/saved_models/sphereface_model.pth"):
  if use_mount:
    !jar xvf "/content/drive/MyDrive/Project 6 - Dataset/saved_models.zip"

if resume_model and not os.path.isfile("/content/saved_models/cosface_model.pth"):
  if use_mount:
    !jar xvf "/content/drive/MyDrive/Project 6 - Dataset/saved_models.zip"

# Download Code

Clone of original repo of CosPlace and our code

In [ ]:
# download code of CosPlace
!git clone "https://github.com/gmberton/CosPlace" 
#!rm -r "/content/CosPlace"

# download our code
!git clone --single-branch --branch "develop" "https://github.com/gab-palmeri/aml-geolocalization.git"
!mv "/content/aml-geolocalization/" "/content/Team"
#!rm -r "/content/aml-geolocalization"



# Import Code


In [ ]:
import os
import sys
import torch
import logging
import multiprocessing
import numpy as np
import torchvision.transforms as T
from tqdm import tqdm
from datetime import datetime

sys.path.append("/content/CosPlace/")
sys.path.append("/content/Team/")
import CosPlace
from CosPlace import *

torch.backends.cudnn.benchmark = True  # Provides a speedup

This class let us to access to dictionary keys like `dict.key` instead of `dict["key"]`

In [ ]:
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

Drive related functions to save and load training checkpoints

In [ ]:
import os
from google.colab import drive

drive.mount('/content/drive')

def drive_save_checkpoint(output_folder, time):
    drive_folder = "/content/drive/MyDrive/project6/"
    if not os.path.exists(drive_folder):
        !mkdir "/content/drive/MyDrive/project6/"
    !zip -r "/content/drive/MyDrive/project6/{time}.zip" "/content/{output_folder}"

def drive_load_checkpoint(input_folder, time):
    drive_folder = "/content/drive/MyDrive/project6/"
    !jar xvf "/content/drive/MyDrive/project6/{time}.zip"

def drive_tester(output_folder):
    drive_folder = "/content/drive/MyDrive/project6/"
    if not os.path.exists(drive_folder):
        !mkdir "/content/drive/MyDrive/project6/"
    !touch "/content/drive/MyDrive/project6/test"

    if os.path.exists("/content/drive/MyDrive/project6/test"):
        logging.info("Drive saving works")
    else:
        logging.info("WARNING: Drive saving does not work")

# use this function when content on drive are not updated
def drive_refresh():
  drive.flush_and_unmount()
  drive.mount('/content/drive')

# Parameters

Original code provides two main executable files: `train.py` and `eval.py`. We want to reproduce the same behaviour of these two files so these are the parameters passed via terminal. They will be put in a dict called `args` to simply reuse code inside ex files

## Setup parameters

In [ ]:
# CosPlace Groups parameters
COS_M = 10
ALPHA = 30
COS_N = 5
COS_L = 2
GROUPS_NUM = 1
MIN_IMAGES_PER_CLASS = 10
# Model parameters
BACKBONE = "convnext_tiny"   
# ["resnet18", "efficientnet_b2", "efficientnet_v2_s", "mobilenet_v3_small", "mobilenet_v3_large",
# ["convnext_tiny", "swin_tiny" ]
FC_OUTPUT_DIM = 512     # Output dimension of final fully connected layer
PRETRAIN = "imagenet"   # ["imagenet", "places", "gldv2"]
# Training parameters
AUGMENTATION_DEVICE = "cuda"    # ["cuda", "cpu"]
USE_AMP_16 = AUGMENTATION_DEVICE == "cuda"       # use Automatic Mixed Precision
BATCH_SIZE = 32
EPOCHS_NUM = 3
ITERATIONS_PER_EPOCH = 10_000
LR = 0.00001                    # Learning rate  
CLASSIFIERS_LR = 0.01
# Data augmentation
RESIZE_AS_DB = False
BRIGHTNESS = 0.7
CONTRAST = 0.7
SATURATION = 0.7
HUE = 0.5
RANDOM_RESIZED_CROP = 0.5
RANDOM_H_FLIP = False
MAJORITY_WEIGHT = 0.01
# Validation / test parameters
INFER_BATCH_SIZE = 16           # Batch size for inference (validating and testing)
POSITIVE_DIST_THRESHOLD = 25    # distance in meters for a prediction to be considered a positive
# Resume parameters
RESUME_TRAIN = None     # path to checkpoint to resume, e.g. logs/.../last_checkpoint.pth
RESUME_MODEL = None     # Path to model to resume training from
# Other parameters
DEVICE = "cuda"                     # ["cuda", "cpu"]
SEED = 0
NUM_WORKERS = 8
DATASET_FOLDER = "/content/small"   # path of the folder with train/val sets
SAVEDIR = BACKBONE + "_" + PRETRAIN
TEST_METHOD = None


if not os.path.exists(DATASET_FOLDER):
    raise FileNotFoundError(f"Dataset folder {DATASET_FOLDER} not found")

train_set_folder = os.path.join(DATASET_FOLDER, "train")

if not os.path.exists(train_set_folder):
    raise FileNotFoundError(f"Train set folder {train_set_folder} not found")

val_set_folder = os.path.join(DATASET_FOLDER, "val")

if not os.path.exists(val_set_folder):
    raise FileNotFoundError(f"Validation set folder {val_set_folder} not found")

if BACKBONE != "resnet18" and PRETRAIN != "imagenet":
    raise ValueError("Only resnet18 can be pretrained on other datasets than ImageNet")

# dictionary for the parameters
args = {
    'M': COS_M, 'alpha': ALPHA, 'N': COS_N, 'L': COS_L, 'groups_num': GROUPS_NUM,
    'min_images_per_class': MIN_IMAGES_PER_CLASS, 'backbone': BACKBONE, "pretrain": PRETRAIN,
    'fc_output_dim': FC_OUTPUT_DIM, 'use_amp16': USE_AMP_16,
    'augmentation_device': AUGMENTATION_DEVICE, 'batch_size': BATCH_SIZE,
    'epochs_num': EPOCHS_NUM, 'iterations_per_epoch': ITERATIONS_PER_EPOCH,
    'lr': LR, 'classifiers_lr': CLASSIFIERS_LR, 'brightness': BRIGHTNESS, 'resize_as_db': RESIZE_AS_DB,
    'contrast': CONTRAST, 'hue': HUE, 'saturation': SATURATION, 'hflip': RANDOM_H_FLIP, 'maj_weight': MAJORITY_WEIGHT,
    'random_resized_crop': RANDOM_RESIZED_CROP, 'infer_batch_size': INFER_BATCH_SIZE,
    'positive_dist_threshold': POSITIVE_DIST_THRESHOLD, 'resume_train': RESUME_TRAIN,
    'resume_model': RESUME_MODEL, 'device': DEVICE, 'seed': SEED,
    'num_workers': NUM_WORKERS, 'dataset_folder': DATASET_FOLDER, 'save_dir': SAVEDIR,
    'val_set_folder': val_set_folder, 'train_set_folder': train_set_folder, 'test_method': TEST_METHOD
}

# this helps to reuse the code from the original CosPlace
args = dotdict(args)


In [ ]:
saveDirLocal = SAVEDIR

# Setup logging

In [ ]:
from CosPlace import commons

start_time = datetime.now()
output_folder = f"logs/{args.save_dir}/{start_time.strftime('%Y-%m-%d_%H-%M-%S')}"
commons.make_deterministic(args.seed)
commons.setup_logging(output_folder, console=None)
logging.info(" ".join(sys.argv))
logging.info(f"Arguments: {args}")
logging.info(f"The outputs are being saved in {output_folder}")

# PAY ATTENTION!
If you want to just test an already provided model, you should skip the training part and go to [Test section](#scrollTo=Y5jJf7v5Ox91)

# Training



## Model

In [ ]:
from Team.model import network

model = network.GeoLocalizationNet(args.backbone, args.fc_output_dim, args.pretrain)

logging.info(f"There are {torch.cuda.device_count()} GPUs and {multiprocessing.cpu_count()} CPUs.")

if args.resume_model is not None:
    logging.debug(f"Loading model from {args.resume_model}")
    model_state_dict = torch.load(args.resume_model)
    model.load_state_dict(model_state_dict)

model = model.to(args.device).train()

## Optimizer

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
model_optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)


## Datasets

### CosFace

In [ ]:
from CosPlace import datasets
from datasets.train_dataset import TrainDataset
from Team.datasets import DataAugTestDataset
from Team.loss.cosface_loss import MarginCosineProduct

groups = [TrainDataset(args, args.train_set_folder, M=args.M, alpha=args.alpha, N=args.N, L=args.L,
                       current_group=n, min_images_per_class=args.min_images_per_class) for n in range(args.groups_num)]

# apply cosface loss to each group
# Each group has its own classifier, which depends on the number of classes in the group
classifiers = [MarginCosineProduct(args.fc_output_dim, len(group)) for group in groups]
classifiers_optimizers = [torch.optim.Adam(classifier.parameters(), lr=args.classifiers_lr) for classifier in classifiers]

logging.info(f"Using {len(groups)} groups")
logging.info(f"The {len(groups)} groups have respectively the following number of classes {[len(g) for g in groups]}")
logging.info(f"The {len(groups)} groups have respectively the following number of images {[g.get_images_num() for g in groups]}")

val_ds = DataAugTestDataset(args.val_set_folder, positive_dist_threshold=args.positive_dist_threshold)
logging.info(f"Validation set: {val_ds}")

## Resume train

In [ ]:
from CosPlace import util

if args.resume_train:
    model, model_optimizer, classifiers, classifiers_optimizers, best_val_recall1, start_epoch_num = \
        util.resume_train(args, output_folder, model, model_optimizer, classifiers, classifiers_optimizers)
    model = model.to(args.device)
    epoch_num = start_epoch_num - 1
    logging.info(f"Resuming from epoch {start_epoch_num} with best R@1 {best_val_recall1:.1f} from checkpoint {args.resume_train}")
else:
    best_val_recall1 = start_epoch_num = 0
    logging.info("Starting from scratch, without resuming from a checkpoint")

drive_tester(output_folder)

## Train and Evaluation Loop

In [ ]:
from CosPlace import augmentations
from Team import test as team_test

logging.info("Start training ...")
logging.info(f"There are {len(groups[0])} classes for the first group, " +
             f"each epoch has {args.iterations_per_epoch} iterations " +
             f"with batch_size {args.batch_size}, therefore the model sees each class (on average) " +
             f"{args.iterations_per_epoch * args.batch_size / len(groups[0]):.1f} times per epoch")

hflip_trans = T.RandomHorizontalFlip() if args.hflip else T.Identity()

if args.augmentation_device == "cuda":
    gpu_augmentation = T.Compose([
            augmentations.DeviceAgnosticColorJitter(brightness=args.brightness,
                                                    contrast=args.contrast,
                                                    saturation=args.saturation,
                                                    hue=args.hue),
            augmentations.DeviceAgnosticRandomResizedCrop([512, 512],
                                                          scale=[1-args.random_resized_crop, 1]),
            hflip_trans,                                                        
            T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])

if args.use_amp16:
    scaler = torch.cuda.amp.GradScaler()

for epoch_num in range(start_epoch_num, args.epochs_num):
    
    #### Train
    epoch_start_time = datetime.now()
    # Select classifier and dataloader according to epoch
    current_group_num = epoch_num % args.groups_num
    classifiers[current_group_num] = classifiers[current_group_num].to(args.device)
    util.move_to_device(classifiers_optimizers[current_group_num], args.device)
    
    dataloader = commons.InfiniteDataLoader(groups[current_group_num], num_workers=args.num_workers,
                                            batch_size=args.batch_size, shuffle=True,
                                            pin_memory=(args.device == "cuda"), drop_last=True)
    
    dataloader_iterator = iter(dataloader)
    model = model.train()
    
    epoch_losses = np.zeros((0, 1), dtype=np.float32)
    for iteration in tqdm(range(args.iterations_per_epoch), ncols=100):
        images, targets, _ = next(dataloader_iterator)
        images, targets = images.to(args.device), targets.to(args.device)
        
        if args.augmentation_device == "cuda":
            images = gpu_augmentation(images)
        

        model_optimizer.zero_grad()
        classifiers_optimizers[current_group_num].zero_grad()
        
        if not args.use_amp16:
            descriptors = model(images)
            output = classifiers[current_group_num](descriptors, targets)
            loss = criterion(output, targets)
            loss.backward()
            epoch_losses = np.append(epoch_losses, loss.item())
            del loss, output, images
            model_optimizer.step()
            classifiers_optimizers[current_group_num].step()
        else:  # Use AMP 16
            with torch.cuda.amp.autocast():
                descriptors = model(images)
                output = classifiers[current_group_num](descriptors, targets)
                loss = criterion(output, targets)
            scaler.scale(loss).backward()
            epoch_losses = np.append(epoch_losses, loss.item())
            del loss, output, images
            scaler.step(model_optimizer)
            scaler.step(classifiers_optimizers[current_group_num])
            scaler.update()
    
    classifiers[current_group_num] = classifiers[current_group_num].cpu()
    util.move_to_device(classifiers_optimizers[current_group_num], "cpu")
    
    logging.debug(f"Epoch {epoch_num:02d} in {str(datetime.now() - epoch_start_time)[:-7]}, "
                  f"loss = {epoch_losses.mean():.4f}")
    
    #### Evaluation
    recalls, recalls_str = team_test.test(args, val_ds, model)
    logging.info(f"Epoch {epoch_num:02d} in {str(datetime.now() - epoch_start_time)[:-7]}, {val_ds}: {recalls_str[:20]}")
    is_best = recalls[0] > best_val_recall1
    best_val_recall1 = max(recalls[0], best_val_recall1)
    # Save checkpoint, which contains all training parameters
    util.save_checkpoint({
        "epoch_num": epoch_num + 1,
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": model_optimizer.state_dict(),
        "classifiers_state_dict": [c.state_dict() for c in classifiers],
        "optimizers_state_dict": [c.state_dict() for c in classifiers_optimizers],
        "best_val_recall1": best_val_recall1
    }, is_best, output_folder)

    # save on drive
    drive_save_checkpoint(output_folder, start_time)


logging.info(f"Trained for {epoch_num+1:02d} epochs, in total in {str(datetime.now() - start_time)[:-7]}")

# Test

## Import model

In [ ]:
# default is the one just trained
resume_model = f"{output_folder}/best_model.pth"

In [ ]:

from CosPlace import datasets
from Team import test as team_test
from Team.model import network
from Team.datasets.test_dataset import DataAugTestDataset

#### Model
model = network.GeoLocalizationNet(args.backbone, args.fc_output_dim, args.pretrain)

logging.info(f"There are {torch.cuda.device_count()} GPUs and {multiprocessing.cpu_count()} CPUs.")

if args.resume_model is not None:
  if os.path.exists(args.resume_model):
    resume_model = args.resume_model

if resume_model is not None:
  if not os.path.exists(resume_model):
    raise FileNotFoundError(f"Model {resume_model} not found.")

  logging.info(f"Loading model from {resume_model}")
  model_state_dict = torch.load(resume_model)
  model.load_state_dict(model_state_dict)
else:
    logging.info("WARNING: You didn't provide a path to resume the model (--resume_model parameter). " +
                 "Evaluation will be computed using randomly initialized weights.")

model = model.to(args.device)

## Test on SF-XS

Test the model on the sf-xs (test) dataset.

In [ ]:
# dataset_folder is the same of the training
test_set_folder = os.path.join(DATASET_FOLDER, "test")
if not os.path.exists(test_set_folder):
    raise FileNotFoundError(f"Test set folder {test_set_folder} not found")

test_ds = DataAugTestDataset(test_set_folder, queries_folder="queries_v1",
                      positive_dist_threshold=args.positive_dist_threshold, test_method = args.test_method, resize=args.resize_as_db)

recalls, recalls_str = team_test.test(args, test_ds, model, args.test_method)
logging.info(f"{test_ds}: {recalls_str}")

# recalls for csv file
sf_xs_r15 = f"{recalls[0]:.1f}/{recalls[1]:.1f}"

## Test on Tokyo-XS

Test the model on the tokyo-xs dataset.

In [ ]:
tokyo_xs_folder = "/content/tokyo_xs"
test_set_folder = os.path.join(tokyo_xs_folder, "test")
if not os.path.exists(test_set_folder):
    raise FileNotFoundError(f"Test set folder {test_set_folder} not found")

test_ds = DataAugTestDataset(test_set_folder,
                      positive_dist_threshold=args.positive_dist_threshold, test_method = args.test_method, resize=args.resize_as_db)

recalls, recalls_str = team_test.test(args, test_ds, model, args.test_method)
logging.info(f"{test_ds}: {recalls_str}")

# recalls for csv file
tokyo_xs_r15 = f"{recalls[0]:.1f}/{recalls[1]:.1f}"

## Test on Tokyo-Night

Test the model on the tokyo-night dataset.

In [ ]:
tokyo_night_folder = "/content/tokyo-night/"
test_set_folder = os.path.join(tokyo_night_folder, "test")
if not os.path.exists(test_set_folder):
    raise FileNotFoundError(f"Test set folder {test_set_folder} not found")

test_ds = DataAugTestDataset(test_set_folder,
                      positive_dist_threshold=args.positive_dist_threshold, test_method = args.test_method, resize=args.resize_as_db)

recalls, recalls_str = team_test.test(args, test_ds, model, args.test_method)
logging.info(f"{test_ds}: {recalls_str}")

# recalls for csv file
tokyo_night_r15 = f"{recalls[0]:.1f}/{recalls[1]:.1f}"

# Save results

## Create CSV with recalls

In [ ]:
import csv
header = ["sf-xs (test)", "Tokyo-xs", "Tokyo-night"]
data = [sf_xs_r15, tokyo_xs_r15, tokyo_night_r15]

for h, d in zip(header, data):
  logging.info(f"{h}: {d}")

with open(f"/content/{SAVEDIR}_{start_time.strftime('%Y-%m-%d_%H-%M-%S')}.csv", "w") as f:
  writer = csv.writer(f)

  writer.writerow(header)
  writer.writerow(data)
logging.info(f"save table results to /content/{SAVEDIR}{start_time.strftime('%Y-%m-%d_%H-%M-%S')}.csv")

## Save on Drive

In [ ]:
modelLocal = SAVEDIR

Save all data generated by this notebook in a specific folder in **PERSONAL** gdrive. Remember to copy inside shared_data of project drive

### Run the following if you did training

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

if not os.path.exists("/content/drive/MyDrive/project6"):
  !mkdir /content/drive/MyDrive/project6
!cp "/content/logs/{SAVEDIR}/{start_time.strftime('%Y-%m-%d_%H-%M-%S')}/best_model.pth" "/content/drive/MyDrive/project6/{modelLocal}"

### Run always

In [ ]:
import os
from google.colab import drive

drive.mount('/content/drive')

# zip logs -> logs.zip
!zip -r /content/drive/MyDrive/project_6/logs.zip /content/logs/
# zip cache -> cache.zip
if os.path.exists("/content/cache/"):
  !zip -r /content/drive/MyDrive/project_6/cache.zip /content/cache/

!cp "/content/{SAVEDIR}_{start_time.strftime('%Y-%m-%d_%H-%M-%S')}.csv" /content/drive/MyDrive/project6/

drive.flush_and_unmount()